## Clasificación TF-IDF

In [1]:
import pandas as pd
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim import models

Lo primero que necesitamos es cargar nuestros glosarios de términos para crear nuestro diccionario

In [25]:
def cargar_glosario(categoria):
    fname = f"../Datos/glosario_{categoria}.txt"
    terminos = []
    with open(fname, 'r') as f:
        termino = f.readline().replace("\n","")
        terminos.append(termino)
    return terminos

In [27]:
categorias = ["deportes", "salud", "ciencia", "politica"]
glosarios = {}
for categoria in categorias:
    glosarios[categoria] = cargar_glosario(categoria)

In [ ]:
glosarios_dict = create_dictionary(glosarios)

Creamos nuestro diccionario de palabras

In [3]:
def create_dictionary(glosarios):
    doc_tokens = [simple_preprocess(corpus) for corpus in dataframe["corpus"]]
    dictionary = corpora.Dictionary(doc_tokens)
    return dictionary

In [4]:
def create_bag_of_words(docs_list, dictionary):
    doc_tokens = [simple_preprocess(corpus) for corpus in docs_list]
    dictionary = corpora.Dictionary(doc_tokens)
    docs_bow = [dictionary.doc2bow(doc, allow_update=True) for doc in doc_tokens]
    return docs_bow

In [5]:
def create_tfidf(docs_bow):
    docs_tfidf = models.TfidfModel(docs_bow, smartirs="lfc")
    return docs_tfidf

In [6]:
noticias_dict = create_dictionary(noticias_test_dataframe)

Ahora es momento de crear una bag of words para cada noticia en base a nuestro diccionario

In [2]:
noticias_test_dataframe = pd.read_csv("../Datos/noticias_test.csv")

In [11]:
docs_bow = create_bag_of_words(noticias_test_dataframe["corpus"].values, noticias_dict)

[(0, 0.15471442329540214), (1, 0.1003955311131111), (2, 0.03557518980057467), (3, 0.1003955311131111), (4, 0.14264881733417556), (5, 0.07132440866708778), (6, 0.05210944191143335), (7, 0.15471442329540214), (8, 0.060986991984878614), (9, 0.05431889218229104), (10, 0.1003955311131111), (11, 0.066645003134445), (12, 0.2595186831673763), (13, 0.1003955311131111), (14, 0.1003955311131111), (15, 0.05431889218229104), (16, 0.066645003134445), (17, 0.066645003134445), (18, 0.07735721164770107), (19, 0.07735721164770107), (20, 0.09010698109161601), (21, 0.07735721164770107), (22, 0.07132440866708778), (23, 0.07132440866708778), (24, 0.1003955311131111), (25, 0.03864952067053617), (26, 0.05210944191143335), (27, 0.05958905176881966), (28, 0.1003955311131111), (29, 0.045053490545808005), (30, 0.07132440866708778), (31, 0.046607563038007234), (32, 0.11917810353763932), (33, 0.03557518980057467), (34, 0.14264881733417556), (35, 0.045053490545808005), (36, 0.1003955311131111), (37, 0.05431889218229

Y con el bag of words podemos crear nuestro modelo tfidf

In [14]:
tfidf_model = create_tfidf(docs_bow)

Cargamos los glosarios y creamos sus representaciones tfidfs